In [ ]:
from openai import OpenAI
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
import time

# Configuração do cliente da API DeepSeek
client = OpenAI(
    api_key="xx",
    base_url="xx"
)

# Carregar dados de teste
df_test = pd.read_csv('campanhas.csv')

# Função para classificar usando DeepSeek
def classify_with_deepseek(h1, h2, cta):
    prompt = f"""
Você é um especialista em CRM que analisa campanhas publicitárias de banners exibidas em um aplicativo de um banco e corretora de investimentos.

Cada campanha possui um título (H1), um subtítulo (H2) e uma chamada para ação (CTA). Seu objetivo é avaliar, com base apenas no conteúdo textual dessas três partes, se a campanha terá desempenho acima da média ou abaixo da média em termos de CTR (taxa de cliques).

A média histórica de CTR nesse canal gira em torno de 0,85%, e é essencial considerar o sentimento, emoção e estrutura semântica dos textos para avaliá-lo.


Exemplos 1

Título (H1): Crédito rápido e fácil
Subtítulo (H2): Taxas reduzidas usando seus investimentos como garantia
CTA: Quero simular!

Hipótese: Campanha utilizou palavras como "rápido" e "fácil" para transmitir um sentimento bom para o cliente, além disso, trouxe uma vantagem com "taxas reduzidas"

Exemplos 2

Título (H1): IPCA + 5,80% a.a.
Subtítulo (H2): Com pagamento de juros semestrais e isenção de IR
CTA: Confira!
CTR: Acima média

Hipótese: Campanha mostra "isenção de IR" como uma vantagem, além de mostrar IPCA +5,8 que pode ser uma opção de investimento boa para o publico alvo dessa campanha

Exemplos 3

Título (H1): Peça seu Cartão Rico!
Subtítulo (H2): Aproveite condições especiais por 30 dias.
CTA: Saiba mais
CTR: Acima média

Hipótese: Campanha traz "condições especiais por 30 dias" para mostrar exclusividade e que escassez

Exemplos 4

Título (H1): 47% de redução em parcela
Subtítulo (H2): Contrate o consórcio para a moto dos seus sonhos.
CTA: Saber mais
CTR: Acima média

Hipótese: Campanha traz um argumento de parcela reduzida, além disso menciona moto como sonho, o que pode fazer sentido para muitos clientes

Exemplos 5

Título (H1): 115% do CDI!
Subtítulo (H2): Produto desbloqueado especialmente para você
CTA: Clique aqui
CTR: Acima média

Hipótese: Campanha traz sentimento bom para o cliente ao indicar que o produto foi desbloqueado especialmente para ele

Exemplos 6

Título (H1): Liquidez e Rentabilidade
Subtítulo (H2): Mantenha uma gestão de caixa eficiente e segura.
CTA: Saiba mais!
CTR: Acima média

Hipótese: Campanha traz as vantagens oferecidas, com Liquidez e Rentabilidade, além do sentimento de segurança e eficiência

Exemplo 7

Título (H1): Abra sua conta PJ!
Subtítulo (H2): Invista no seu negócio pela XP!
CTA: Falar com assessor!
CTR: Abaixo média

Exemplo 8

Título (H1): WEBINAR XP
Subtítulo (H2): Saiba sobre as perspectivas de mercado para 2024
CTA: Acessar Live
CTR: Abaixo média

Exemplo 9

Título (H1): Segurança e retorno!
Subtítulo (H2): Cuide do seu futuro com a Previdência Az Quest XP.
CTA: Saiba mais
CTR: Abaixo média

Exemplo 10

Título (H1): Abra sua conta PJ!
Subtítulo (H2): Invista no seu negócio pela XP!
CTA: Saiba mais!
CTR: Abaixo média

Agora, com base nesse histórico de campanhas/resultados, analisando o tom da mensagem, o sentimento/emoção passado e o contexto da campanha, preciso que classifique o seguinte conteúdo:

Título (H1): {h1}
Subtítulo (H2): {h2}
CTA: {cta}

Responda apenas com: Acima ou Abaixo.
"""

    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "Você é um especialista em CRM"},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=5
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Erro na requisição: {e}")
        return "Erro"

# Lista para armazenar predições
predictions = []

# Loop com controle de rate limit
for index, row in df_test.iterrows():
    prediction = classify_with_deepseek(row['H1'], row['H2'], row['CTA'])
    predictions.append(prediction)
    time.sleep(21)  # ajuste com base no limite da API

# Avaliar resultados
df_test['predictions'] = predictions
df_results = df_test[df_test['predictions'].isin(['Acima', 'Abaixo'])]

# Mapeia predições para valores binários
mapa_pred = {'Acima': 1, 'Abaixo': 0}
df_results['pred_bin'] = df_results['predictions'].map(mapa_pred)

# Exibe métricas
print(classification_report(df_results['ACIMA_MEDIA'], df_results['pred_bin']))
print('Acurácia:', accuracy_score(df_results['ACIMA_MEDIA'], df_results['pred_bin']))
